<a href="https://colab.research.google.com/github/Luck1e23/STA160-Team-11-Project/blob/main/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torchxrayvision
!pip install iterative-stratification
!pip install efficientnet_pytorch

### Packages

In [ ]:
from PIL import Image
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
from torch.optim import Adam
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
from pathlib import Path
from sklearn.metrics import f1_score, roc_auc_score, precision_recall_curve

# Visualization tools
import torchvision
import torchvision.transforms.v2 as transforms
import torch.nn.functional as F
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Pre-trained Model: torchxrayvision
import torchxrayvision as xrv
import skimage

# Pre-trained Modedl: EfficientNet
from efficientnet_pytorch import EfficientNet


### Pre-Trained Model

In [ ]:
# XRV Pathology Classifiers :: NIH chest X-ray8
xrv_model = xrv.models.DenseNet(weights="densenet121-res224-nih")

### File Paths

In [ ]:
# Paths
file_path = '/content/drive/Shareddrives/STA_160/dataset/Data_Entry_2017.csv'
train_val = '/content/drive/Shareddrives/STA_160/dataset/train_val_list.txt'
test = '/content/drive/Shareddrives/STA_160/dataset/test_list.txt'
resized_root = '/content/dataset_resized'   # Where resized images will be saved

### Creating a 224 x 224 Resized Dataset

In [ ]:
# Old Code for resizing the images before making a dataset class
# Purpose was to speed up the running of the code

# Target Size
IMG_SIZE = (224, 224)

for root, dirs, files in os.walk(dataset_root):
    rel_path = os.path.relpath(root, dataset_root)
    save_dir = os.path.join(resized_root, rel_path)
    os.makedirs(save_dir, exist_ok=True)

    for f in files:
        if f.lower().endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(root, f)

            # Load and convert to grayscale (preserve pixel values)
            img = Image.open(img_path).convert("L")
            arr = np.array(img).astype(np.uint8)

            # Resize using bilinear interpolation
            img_resized = Image.fromarray(arr).resize(IMG_SIZE, Image.BILINEAR)

            # Build output filename (always PNG)
            base = os.path.splitext(f)[0]
            out_path = os.path.join(save_dir, base + ".png")

            # Save as PNG to avoid JPEG compression artifacts
            img_resized.save(out_path, format="PNG")

# Zip the resized dataset
!zip -r -q /content/NIH_resized.zip /content/dataset_resized

# Copy and upload to the shared drive
!cp /content/NIH_resized.zip /content/drive/Shareddrives/STA_160/



In [ ]:
# Unzipping the resized dataset from shared drives
!unzip -q /content/drive/Shareddrives/STA_160/NIH_resized.zip -d /content/dataset

!mkdir -p /content/dataset_resized
# Finds all image files inside subfolders
!find /content/dataset/content/dataset_resized/ -type f -exec mv -t /content/dataset_resized/ {} +
!rm -rf /content/dataset/content #remove folder

### Dataset

In [ ]:
# Dataset Class
class NIHXrays(Dataset):
    def __init__(self, file_path, dataset_root, list_file=None, transform=None):
        self.data = pd.read_csv(file_path)
        self.dataset_root = dataset_root
        self.transform = transform

        # Optional filtering
        if list_file:
            with open(list_file, 'r') as f:
                image_list = {line.strip() for line in f.readlines()}
            self.data = self.data[self.data['Image Index'].isin(image_list)].reset_index(drop=True)

        # Create label map
        all_labels = set()
        for labels in self.data['Finding Labels']:
            for l in labels.split('|'):
                all_labels.add(l.strip())

        self.all_labels = sorted(all_labels)
        self.label_map = {label: i for i, label in enumerate(self.all_labels)}

        # Build multi-hot label vectors
        self.finding_labels = []
        for labels in self.data['Finding Labels']:
            vec = torch.zeros(len(self.all_labels))
            for l in labels.split('|'):
                if l.strip() in self.label_map:
                    vec[self.label_map[l.strip()]] = 1.0
            self.finding_labels.append(vec)

        self.finding_labels = torch.stack(self.finding_labels).float()

        # Recursively map image filenames to full paths
        self.image_map = {}
        for img_path in Path(dataset_root).rglob("*.png"):
            self.image_map[img_path.name] = str(img_path)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = self.data.iloc[idx]['Image Index']

        if img_name not in self.image_map:
            raise FileNotFoundError(f"Image {img_name} not found.")

        # Load Image
        img = Image.open(self.image_map[img_name]).convert("L")

        if self.transform is not None:
          img = self.transform(img)

        img = np.array(img).astype(np.float32) # PIL image --> NumPy

        # XRV normalizaiton
        img = xrv.datasets.normalize(img, maxval=255)

        #   NumPy --> Tensor
        img = torch.from_numpy(img).unsqueeze(0)

        label = self.finding_labels[idx].float()

        return img, label, img_name

In [ ]:
# Data Augmentation
rand_transforms = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip()
])

train_val_data = NIHXrays(file_path, resized_root, list_file=train_val, transform=rand_transforms)
test_data      = NIHXrays(file_path, resized_root, list_file=test, transform=None)

### Multi-Label Stratified Split for Training and Validation

In [ ]:
# Convert labels to NumPy for the stratifier
y = train_val_data.finding_labels.numpy()   # shape: [N, C]
X = np.arange(len(train_val_data))          # dummy feature array

#Stratified Splitting
msss = MultilabelStratifiedShuffleSplit(
    n_splits=1,
    test_size=0.2,
    random_state=42
)
#only want the first split & get train and validation indices
train_idx, valid_idx = next(msss.split(X, y))

# Get train and validation datasets based on the indices
train_data = Subset(train_val_data, train_idx)
valid_data = Subset(train_val_data, valid_idx)

### Data Augmentation and Data Loader

In [ ]:
# Create DataLoaders for training and validation
n = 32
train_loader = DataLoader(train_data, batch_size=n, shuffle=True, num_workers=2, pin_memory=True)
train_N = len(train_loader.dataset)
valid_loader = DataLoader(valid_data, batch_size=n, num_workers=2, pin_memory=True)
valid_N = len(valid_loader.dataset)